In [1]:
import torch
from d2l import torch as d2l

In [2]:
def corr2d_multi_in(X, K):
    return sum(d2l.corr2d(x, k) for x, k in zip(X, K))

K = torch.stack((K, K + 1, K + 2), 0)
`zip(X, K)`将每一个通道和相应的卷积核进行绑定，生成一个元组，然后遍历每一个元组，分别进行corr2d操作，再将结果相加得到输出通道为1的结果

In [3]:
X = torch.tensor([  [[0.0, 1.0, 2.0],
                     [3.0, 4.0, 5.0],
                     [6.0, 7.0, 8.0]],

                    [[1.0, 2.0, 3.0],
                     [4.0, 5.0, 6.0],
                     [7.0, 8.0, 9.0]]]) 
                    # 输入通道数为2，整体形状为(2,3,3)

K = torch.tensor([  [[0.0, 1.0], 
                     [2.0, 3.0]],

                    [[1.0, 2.0], 
                     [3.0, 4.0]]])
                     # 输入通道数为2，整体形状为(2,2,2)

corr2d_multi_in(X, K)

tensor([[ 56.,  72.],
        [104., 120.]])

In [4]:
def corr2d_multi_in_out(X, K):
    return torch.stack([corr2d_multi_in(K, k) for k in K], 0)

K = torch.stack((K, K + 1, K + 2), 0)
K.shape

torch.Size([3, 2, 2, 2])

`torch.stack()`是 PyTorch 中一个非常重要的张量操作函数。它的作用是：将一个序列（比如列表或元组）中的多个张量沿着一个 `新创建` 的维度进行堆叠，从而合并成一个更高维度的张量。
`0`: 这个参数是 dim（dimension，维度）。dim=0 的意思是：“在第 0 维的位置上创建一个新的维度来进行堆叠”
堆叠出来的是四维向量，由于dim=0，因此堆叠出来的向量的形状为(3, 2, 2, 2)

In [5]:
print(K)

tensor([[[[0., 1.],
          [2., 3.]],

         [[1., 2.],
          [3., 4.]]],


        [[[1., 2.],
          [3., 4.]],

         [[2., 3.],
          [4., 5.]]],


        [[[2., 3.],
          [4., 5.]],

         [[3., 4.],
          [5., 6.]]]])


In [6]:
def corr2d_multi_in_out_1x1(X, K):
    c_i, h, w = X.shape
    c_o = K.shape[0]
    X = X.reshape((c_i, h * w))
    K = K.reshape((c_o, c_i))
    Y = torch.matmul(K, X)
    return Y.reshape((c_o, h, w))


In [7]:
X = torch.normal(0, 1, (3, 3, 3))
K = torch.normal(0, 1, (2, 3, 1, 1))
Y1 = corr2d_multi_in_out_1x1(X, K)
Y2 = corr2d_multi_in_out(X, K)
assert float(torch.abs(Y1 - Y2).sum()) < 1e-6

AssertionError: 